In [1]:
import pandas as pd
import datetime
import numpy as np

In [19]:
smiles_df = pd.read_csv('Polymer-SMILES-FP.csv').drop_duplicates('SMILES')
smiles_df.head(10)

,index,SMILES,Polymer,afp_C4_C4_C4,afp_C4_C4_F1,afp_F1_C4_F1,bfp_299,bfp_304,bfp_328,bfp_334,...,bfp_373,bfp_378,bfp_429,afp_C3_C3_N3,afp_C3_N3_C3,bfp_46,bfp_47,bfp_55,bfp_79,bfp_210
0,0,[*]C([*])(F)F,Polytetrafluoroethylene,0.333333,1.333333,0.333333,1.0,1.000000,1.333333,0.666667,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
1,1,[*]C=CCC[*],polybutadiene,0.000000,0.000000,0.000000,0.0,0.600000,0.000000,0.600000,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
2,2,[*]CC(C)O[*],"poly(propane-1,2-diol)",0.100000,0.000000,0.000000,0.0,0.300000,0.000000,0.400000,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
3,3,[*]CC([*])(C)C,polyisobutylene,0.583333,0.000000,0.000000,0.0,1.750000,0.000000,0.666667,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
5,5,[*]CC([*])(C)C(=O)OC,poly(methyl methacrylate),0.266667,0.000000,0.000000,0.0,1.400000,0.000000,0.533333,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
6,6,[*]CC([*])(F)F,poly(vinylidene fluoride),0.333333,0.666667,0.166667,0.5,1.000000,0.666667,0.666667,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
7,7,[*]CC([*])C,polypropylene,0.444444,0.000000,0.000000,0.0,1.333333,0.000000,0.666667,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
9,9,[*]CC([*])C#N,polyacrylonitrile,0.285714,0.000000,0.000000,0.0,1.714286,0.000000,0.857143,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
10,10,[*]CC([*])CC,Polybutene,0.416667,0.000000,0.000000,0.0,1.250000,0.000000,0.666667,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0
11,11,[*]CC([*])CC(C)C,poly(4-methyl-1-pentene),0.444444,0.000000,0.000000,0.0,1.333333,0.000000,0.666667,...,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0


In [20]:
data = pd.read_excel('Melt Viscosity Data.xlsx', ["Homopolymer", "Copolymer", "Polymer Blend"])
data["Copolymer"] = data["Copolymer"].drop(columns = ["INDEX", "PID"])
#data['Copolymer'].loc[0, data["Copolymer"].columns.isin(fp_columns)]

In [21]:
#make new FP for CoPolymers
fp_columns = smiles_df.columns[3:]
for i in data["Copolymer"].index:
    s_1 = data["Copolymer"].loc[i,"SMILES 1"]
    s_2 = data["Copolymer"].loc[i,"SMILES 2"]
    w_1 = data["Copolymer"].loc[i,"Weight 1"]
    w_2 = data["Copolymer"].loc[i,"Weight 2"]
    fp_1 = smiles_df.loc[smiles_df["SMILES"] == s_1, list(fp_columns.values)]
    fp_2 = smiles_df.loc[smiles_df["SMILES"] == s_2, list(fp_columns.values)]
    fp = (fp_1*w_1).add(fp_2*w_2, fill_value = 0).sum()
    data["Copolymer"].loc[i, fp_columns] = fp

#Make new FP for Polymer Blends
for i in data["Polymer Blend"].index:
    s_1 = data["Polymer Blend"].loc[i,"SMILES 1"]
    s_2 = data["Polymer Blend"].loc[i,"SMILES 2"]
    w_1 = data["Polymer Blend"].loc[i,"Weight 1"]
    w_2 = data["Polymer Blend"].loc[i,"Weight 2"]
    fp_1 = smiles_df.loc[smiles_df["SMILES"] == s_1, list(fp_columns.values)]
    fp_2 = smiles_df.loc[smiles_df["SMILES"] == s_2, list(fp_columns.values)]
    fp_prod = (fp_1.reset_index(drop = True)*fp_2.reset_index(drop = True)).loc[0,:]
    fp = 1/((w_2*fp_1).add((w_1*fp_2), fill_value = 0).sum())
    fp = (fp_prod*fp).fillna(0)
    data["Polymer Blend"].loc[i, fp_columns] = fp

In [22]:
data["Copolymer"]["Polymer 1"] = (data["Copolymer"]["Polymer 1"]  + ", " + data["Copolymer"]["Polymer 2"])
data["Copolymer"]["SMILES 1"] = (data["Copolymer"]["SMILES 1"] + ", " + data["Copolymer"]["SMILES 2"])
data["Copolymer"] = data["Copolymer"].rename(columns = {"Polymer 1": "Polymer", "SMILES 1": "SMILES"}).drop(columns = ["Polymer 2","SMILES 2"]).reset_index(drop=True)
data["Copolymer"]

,Unnamed: 0,SAMPLE_ID,SMILES,Polymer,Weight 1,Weight 2,Mn,Mw,PDI,Temperature,...,bfp_373,bfp_378,bfp_429,afp_C3_C3_N3,afp_C3_N3_C3,bfp_46,bfp_47,bfp_55,bfp_79,bfp_210
0,0,NaN,"[*]CC([*])c1ccccc1, [*]CC([*])C#N","polystyrene, polyacrylonitrile",0.94,0.06,80000.0,160000,NaN,170,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,NaN,"[*]CC([*])c1ccccc1, [*]CC([*])C#N","polystyrene, polyacrylonitrile",0.87,0.13,NaN,100000,NaN,170,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,NaN,"[*]CC([*])c1ccccc1, [*]CC([*])C#N","polystyrene, polyacrylonitrile",0.81,0.19,NaN,60000,NaN,170,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,NaN,"[*]CC([*])c1ccccc1, [*]CC([*])C#N","polystyrene, polyacrylonitrile",0.76,0.24,53000.0,106000,NaN,170,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,NaN,"[*]CC([*])c1ccccc1, [*]CC([*])C#N","polystyrene, polyacrylonitrile",0.72,0.28,56000.0,112000,NaN,170,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,239,NaN,"[*]CC([*])C#N, [*]CC([*])C(=O)OC","polyacrylonitrile, Poly(methyl acrylate)",0.90,0.10,NaN,33000,NaN,220,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
240,240,NaN,"[*]CC([*])C#N, [*]CC([*])C(=O)OC","polyacrylonitrile, Poly(methyl acrylate)",0.90,0.10,NaN,33000,NaN,220,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
241,241,NaN,"[*]CC([*])C#N, [*]CC([*])C(=O)OC","polyacrylonitrile, Poly(methyl acrylate)",0.90,0.10,NaN,33000,NaN,220,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
242,242,NaN,"[*]CC([*])C#N, [*]CC([*])C(=O)OC","polyacrylonitrile, Poly(methyl acrylate)",0.90,0.10,NaN,33000,NaN,220,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
data["Polymer Blend"]["Polymer 1"] = (data["Polymer Blend"]["Polymer 1"]  + ", " + data["Polymer Blend"]["Polymer 2"])
data["Polymer Blend"]["SMILES 1"] = (data["Polymer Blend"]["SMILES 1"] + ", " + data["Polymer Blend"]["SMILES 2"])
data["Polymer Blend"] = data["Polymer Blend"].rename(columns = {"Polymer 1": "Polymer", "SMILES 1": "SMILES"}).drop(columns = ["Polymer 2","SMILES 2"]).reset_index(drop=True)
data["Polymer Blend"]["Mw"] = data["Polymer Blend"]["Mw 1"]*data["Polymer Blend"]["Weight 1"] + data["Polymer Blend"]["Mw 2"]*data["Polymer Blend"]["Weight 2"]
data["Polymer Blend"] = data["Polymer Blend"].drop(columns = ["Mw 1", "Mw 2"])
data["Polymer Blend"]

,Unnamed: 0,INDEX,SAMPLE_ID,PID,SMILES,Polymer,Weight 1,Weight 2,PDI,Temperature,...,bfp_378,bfp_429,afp_C3_C3_N3,afp_C3_N3_C3,bfp_46,bfp_47,bfp_55,bfp_79,bfp_210,Mw
0,0,NaN,NaN,NaN,"[*]CCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc(C([*]...","Poly(hexano-6-lactam), PPTA",0.975,0.025,NaN,260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37075.0
1,1,NaN,NaN,NaN,"[*]CCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc(C([*]...","Poly(hexano-6-lactam), PPTA",0.975,0.025,NaN,260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37075.0
2,2,NaN,NaN,NaN,"[*]CCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc(C([*]...","Poly(hexano-6-lactam), PPTA",0.975,0.025,NaN,260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37075.0
3,3,NaN,NaN,NaN,"[*]CCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc(C([*]...","Poly(hexano-6-lactam), PPTA",0.975,0.025,NaN,260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37075.0
4,4,NaN,NaN,NaN,"[*]CCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc(C([*]...","Poly(hexano-6-lactam), PPTA",0.975,0.025,NaN,260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37075.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,NaN,NaN,NaN,"[*]CCCCCCCCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc...","poly(dodecano-12-lactam), PPTA",0.700,0.300,NaN,200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34400.0
63,63,NaN,NaN,NaN,"[*]CCCCCCCCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc...","poly(dodecano-12-lactam), PPTA",0.700,0.300,NaN,200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34400.0
64,64,NaN,NaN,NaN,"[*]CCCCCCCCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc...","poly(dodecano-12-lactam), PPTA",0.700,0.300,NaN,200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34400.0
65,65,NaN,NaN,NaN,"[*]CCCCCCCCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc...","poly(dodecano-12-lactam), PPTA",0.700,0.300,NaN,200,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34400.0


In [25]:
data["Homopolymer"] = data["Homopolymer"].merge(smiles_df, how = 'left',on = 'SMILES').rename(columns = {"Polymer_x": "Polymer"}).drop(columns = ["INDEX", "PID"])

In [26]:
out = pd.concat([data["Homopolymer"], data["Copolymer"], data["Polymer Blend"]], ignore_index = True)
out = out.rename(columns = {"Melt Viscosity (poise)":"Melt_Viscosity", "Shear Rate":"Shear_Rate"}).rename_axis("INDEX")
#out.loc[,"Mw"] = 

In [27]:
#out["Mw"] = out["Mw"].fillna(np.nan)
#print(out["Mw"].isnull())
#out.loc[out["Mw"].isnull()] =
out.loc[out["Mw"].isnull() & out["Mn"].notnull(), "Mw"] = out.loc[out["Mw"].isnull() & out["Mn"].notnull() , "Mn"]*2

In [28]:
out

,SAMPLE_ID,SMILES,Polymer,Mn,Mw,PDI,PDI (zw),Temperature,Shear_Rate,Frequency,...,bfp_46,bfp_47,bfp_55,bfp_79,bfp_210,Unnamed: 0,Weight 1,Weight 2,INDEX,PID
INDEX,,,,,,,,,,,,,,,,,,,,,
0,1.0,[*]CC([*])C,poly(prop-1-ene),2600.000000,3900.0,1.50,NaN,200.0,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,1.0,[*]CC([*])C,poly(prop-1-ene),7834.101382,17000.0,2.17,NaN,200.0,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,1.0,[*]CC([*])C,poly(prop-1-ene),31981.981982,71000.0,2.22,NaN,200.0,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,1.0,[*]CC([*])C,poly(prop-1-ene),43829.787234,103000.0,2.35,NaN,200.0,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,1.0,[*]CC([*])C,poly(prop-1-ene),75757.575758,250000.0,3.30,NaN,200.0,0.000000,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,NaN,"[*]CCCCCCCCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc...","poly(dodecano-12-lactam), PPTA",NaN,34400.0,NaN,NaN,200.0,115.404361,NaN,...,0.0,0.0,0.0,0.0,0.0,62.0,0.7,0.3,NaN,NaN
998,NaN,"[*]CCCCCCCCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc...","poly(dodecano-12-lactam), PPTA",NaN,34400.0,NaN,NaN,200.0,192.506077,NaN,...,0.0,0.0,0.0,0.0,0.0,63.0,0.7,0.3,NaN,NaN
999,NaN,"[*]CCCCCCCCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc...","poly(dodecano-12-lactam), PPTA",NaN,34400.0,NaN,NaN,200.0,386.070543,NaN,...,0.0,0.0,0.0,0.0,0.0,64.0,0.7,0.3,NaN,NaN


In [29]:
out.to_excel('Data/full_data_' + str(datetime.date.today()) +'.xlsx')

In [124]:
zero_shear_data = out.loc[out["Shear_Rate"] == 0,:]
shear_data = out.loc[out["Shear_Rate"] != 0,:]

In [125]:
zero_shear_data.to_excel('Data/zero_shear_data_' + str(datetime.date.today()) + '.xlsx')
shear_data.to_excel('Data/shear_data_'+ str(datetime.date.today()) +'.xlsx')

In [122]:
str(datetime.date.today())

'2021-11-05'